In [1]:
# IMPORTS
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
import private

In [2]:
# Set some display options for Pandas.
pd.set_option('display.max_colwidth', -1)
pd.set_option('expand_frame_repr', False)

/var/folders/_s/_4ktyf8904n71s9jshmj85500000gn/T/ipykernel_10048/3998464097.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
##  MONTHLY FINANCIAL STATEMENT

#####  CONNECTION ####

def get_statement(coin, period):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = f'{private.first}{coin}{private.second}'
    driver.get(url)

    driver.implicitly_wait(10)


    rows = driver.find_elements(By.CSS_SELECTOR, private.CSS_CLASS)

    rows = [row.text for row in rows]

    rows_clean = []
    earnings = []

    for row in rows:
        row_values = row.split("\n")
        row_clean = []
        for index, value in enumerate(row_values):
            if index % 2 == 1:
                row_clean.append(value)
        row_clean.insert(0, row_values[0])
        rows_clean.append(row_clean)
    

    df = pd.DataFrame(rows_clean[1:], columns=rows_clean[0]).set_index(rows_clean[0][0])
    monthly_df = df[:]

    # Transpose the dataframe to have dates as index
    df = df.T

    # Convert index to datetime format
    df.index = pd.to_datetime(df.index)

    def convert_to_numeric(val):
        if pd.isna(val) or val is None or val == 'N/A':
            return float('nan')
        if isinstance(val, (int, float)):
            return float(val)
        if '%' in val: 
            return val

        # Adjust the regex to consider 'm', 'b', 'k' even without the dollar sign
        match = re.match(r'\$?(-?\d+\.?\d*)([mbkx]?)', val)
        if match is None:
            try:
                return float(val)
            except ValueError:
                return float('nan')

        number = float(match.group(1))
        multiplier = match.group(2)
        if multiplier.lower() == 'b':
            number *= 1e9
        elif multiplier.lower() == 'm':
            number *= 1e6
        elif multiplier.lower() == 'k':
            number *= 1e3
        return number

    def convert_to_string(val):
        if isinstance(val, str) or pd.isna(val):  # check for NaN values
            return val  # return the value as is if it's a percentage or NaN
        sign = '-' if val < 0 else ''
        val = abs(val)
        if val >= 1e9:
            return f'{sign}${val/1e9:.1f}b'
        elif val >= 1e6:
            return f'{sign}${val/1e6:.1f}m'
        elif val >= 1e3:
            return f'{sign}${val/1e3:.1f}k'
        else:
            return f'{sign}${val:.1f}'

    # Apply conversion to all columns
    for column in df.columns:
        df[column] = df[column].apply(convert_to_numeric)

    quarterly_df = df.resample('Q').sum()
    yearly_df = df.resample('Y').sum()
    
    ### EARNINGS LIST
    earnings = yearly_df.T.loc['Earnings'].tolist()  # Use the index of the row you want

    # Convert quarterly and yearly df sums back to string format
    for column in quarterly_df.columns:
        quarterly_df[column] = quarterly_df[column].apply(convert_to_string)
    for column in yearly_df.columns:
        yearly_df[column] = yearly_df[column].apply(convert_to_string)

    columns_remove_dollar = ['Active users (daily)', 'Active users (weekly)', 'Active users (monthly)', 'Core developers', 'Code commits']

    for col in columns_remove_dollar:
        quarterly_df[col] = quarterly_df[col].str.replace('^\$', '', regex=True)
        yearly_df[col] = yearly_df[col].str.replace('^\$', '', regex=True)

    columns_to_drop = ['Price', 'Market cap (fully diluted)', 'Market cap (circulating)', 'P/F ratio (fully diluted)','P/F ratio (circulating)', 'P/S ratio (fully diluted)','P/S ratio (circulating)']
    quarterly_df = quarterly_df.drop(columns=columns_to_drop)
    yearly_df = yearly_df.drop(columns=columns_to_drop)


    quarterly_df = quarterly_df.T
    yearly_df = yearly_df.T

    # yearly_df headers
    for date in list(yearly_df.columns):
        yearly_df = yearly_df.rename(columns={date: str(date)[:4]})
    
    if period == 'y':
        display(yearly_df)
    if period == 'q':
        display(quarterly_df)
    if period == 'm':
        display(monthly_df)
    
    driver.quit()
    return earnings

def get_cumulative_earnings(earnings):
    return sum(earnings[-3:-1])

def get_TVL(coin_TVL):
    url = f'https://api.llama.fi/tvl/{coin_TVL}'
    return requests.get(url).text


def main():
    coin = 'maiar'
    period = 'y'
    earnings = get_statement(coin,period)
    print(' '*15 + str(earnings))

if __name__ == "__main__":
    main()


/var/folders/_s/_4ktyf8904n71s9jshmj85500000gn/T/ipykernel_10048/1457371453.py:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  quarterly_df = df.resample('Q').sum()
/var/folders/_s/_4ktyf8904n71s9jshmj85500000gn/T/ipykernel_10048/1457371453.py:84: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly_df = df.resample('Y').sum()


,2021,2022,2023
Financial statement,,,
Trading volume,$1.3b,$2.0b,$336.4m
Fees,$3.9m,$6.1m,$1.0m
Supply-side fees,$3.2m,$5.1m,$850.7k
Revenue,$649.0k,$1.0m,$170.1k
Token incentives,$0.0,$0.0,$0.0
Earnings,$0.0,$0.0,$0.0
Treasury,$0.0,$0.0,$0.0
MEX trading volume,$434.3m,$345.7m,$21.1m
MEX tokenholders,$0.0,$0.0,$0.0


               [0.0, 0.0, 0.0]
